In [ ]:
import sys


no_of_cpu = 8
max_cores = 16
executor_mem = '56g'



Job_Name = 'Dev Nandan : Customer_Address'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'gpanalytics'
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



In [ ]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 



In [ ]:
def load_gpdb_jdbc(col_str,dbtable,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

In [ ]:
def get_latest_progress(table_name):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')==table_name)\
            .filter(col('source')=='home').filter(col('records')!='0')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2018, 10, 31, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

In [ ]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        
                
#         df_progress_values = [{"table_name":table_name,"source":source,"from_datetime":time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S")
#                         ,"to_datetime":time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),'records':records,
#                          'start_time':start_time}]

        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
        
#         schema = StructType([StructField("table_name", StringType(),True),StructField("source", StringType(),True),\
#                      StructField("from_datetime", StringType(),True),StructField("to_datetime", StringType(),True),\
#                      StructField("records", IntegerType(),True),StructField("start_time", StringType(),True)])

#         df_progress = sqlContext.createDataFrame(df_progress_values,schema)

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

In [ ]:
table_dict={
    "address" : #source/subsegment
              {
                  "out_columns" : ['source_system_name', 'source_system_customer_id','activity_id','activity_date',
                                   'address_type',
                                   'address_id','address_line_1','address_line_2','address_line_3','area','city',
                                   'district','state',
                                   'country','pin_code','landmark','verification_flag', 'verification_source', 'last_updated'],
                  "primary_key" : ['source_system_name','source_system_customer_id','activity_id'],
                  "date_sort_column": "last_updated",
                  "home":
                     {
                          "schema":"public",
                          "table": "mastercraft_bulk_detail_prod",                         
                          "update_fields" : ['verification_flag', 'verification_source', 'last_updated'],
                          "in_columns" : ['source_id','source_customer_id','dc_unified_id','dc_action_date','psnt_address_line_1','psnt_address_line_2','psnt_address_line_3','psnt_address_city','psnt_address_state','psnt_address_country','psnt_address_zip','psnt_address_landmark'],
                          "required_columns" : ['dc_unified_id','source_id'],
                          "date_column" : "dc_action_date",
                          "transform_expr":[
                                "coalesce(source_customer_id,dc_unified_id) as source_system_customer_id",
                                "source_id as source_system_name",
                                "hash(concat_ws(' ',psnt_address_line_1,psnt_address_line_2,psnt_address_line_3,psnt_address_city,psnt_address_state,psnt_address_country,psnt_address_zip,psnt_address_landmark)) as activity_id",
                                "dc_action_date as activity_date",
                                "'home' as address_type",
                                "hash(concat_ws(' ',psnt_address_line_1,psnt_address_line_2,psnt_address_line_3,psnt_address_city,psnt_address_country,psnt_address_zip,psnt_address_landmark)) as address_id",
                                "psnt_address_line_1 as address_line_1",
                                "psnt_address_line_2 as address_line_2",
                                "psnt_address_line_3 as address_line_3",
                                "null as area",
                                "psnt_address_city as city",
                                "psnt_address_dist as district",
                                "psnt_address_state as state",
                                "psnt_address_country as country",
                                "psnt_address_zip as pin_code",
                                "psnt_address_landmark as landmark",
                                "case when psnt_address_is_verified is null then 'UV' else 'IV' end as verification_flag",
                                "case when psnt_address_is_verified is null then null else 'Mastercraft' end as verification_source",
                                "dc_action_date as last_updated",
                                "null as address_json"] 
                     },
                "communication": #source/subsegment
                     {
                          "schema":"public",
                          "table": "mastercraft_bulk_detail_prod",                         
                          "update_fields" : ['verification_flag', 'verification_source', 'last_updated'],
                          "in_columns" : ['source_id','source_customer_id','dc_unified_id','dc_action_date','comm_address_line_1','comm_address_line_2',
                                         'comm_address_line_3','comm_address_city','comm_address_dist','comm_address_state',
                                         'comm_address_country','comm_address_zip','comm_address_landmark','comm_address_is_verified'],
                          "required_columns" : ['dc_unified_id','source_id'],
                          "date_column" : "dc_action_date",
                          "transform_expr":[
                                "coalesce(source_customer_id,dc_unified_id) as source_system_customer_id",
                                "source_id as source_system_name",
                                "hash(concat_ws(' ',comm_address_line_1,comm_address_line_2,comm_address_line_3,comm_address_city,comm_address_dist,comm_address_state,comm_address_country,comm_address_zip,comm_address_landmark)) as activity_id",
                                "dc_action_date as activity_date",
                                "'communication' as address_type",
                                "hash(concat_ws(' ',comm_address_line_1,comm_address_line_2,comm_address_line_3,comm_address_city,comm_address_dist,comm_address_state,comm_address_country,comm_address_zip,comm_address_landmark)) as address_id",
                                "comm_address_line_1 as address_line_1",
                                "comm_address_line_2 as address_line_2",
                                "comm_address_line_3 as address_line_3",
                                "null as area",
                                "comm_address_city as city",
                                "comm_address_dist as district",
                                "comm_address_state as state",
                                "comm_address_country as country",
                                "comm_address_zip as pin_code",
                                "comm_address_landmark as landmark",
                                "case when comm_address_is_verified is null then 'UV' else 'IV' end as verification_flag",
                                "case when comm_address_is_verified is null then null else 'Mastercraft' end as verification_source",
                                "dc_action_date as last_updated",
                                "null as address_json"] 
                     },
                "permanent": #source/subsegment
                     {
                          "schema":"public",
                          "table": "mastercraft_bulk_detail_prod",                         
                          "update_fields" : ['verification_flag', 'verification_source', 'last_updated'],
                          "in_columns" : ['source_id','source_customer_id','dc_unified_id','dc_action_date','pmt_address_line_1','pmt_address_line_2',
                                         'pmt_address_line_3','pmt_address_city','pmt_address_dist','pmt_address_state',
                                         'pmt_address_country','pmt_address_zip','pmt_address_landmark','pmt_address_is_verified'],
                          "required_columns" : ['dc_unified_id','source_id'],
                          "date_column" : "dc_action_date",
                          "transform_expr":[
                                "coalesce(source_customer_id,dc_unified_id) as source_system_customer_id",
                                "source_id as source_system_name",
                                "hash(concat_ws(' ',pmt_address_line_1,pmt_address_line_2,pmt_address_line_3,pmt_address_city,pmt_address_dist,pmt_address_state,pmt_address_country,pmt_address_zip,pmt_address_landmark)) as activity_id",
                                "dc_action_date as activity_date",
                                "'permanent' as address_type",
                                "hash(concat_ws(' ',pmt_address_line_1,pmt_address_line_2,pmt_address_line_3,pmt_address_city,pmt_address_dist,pmt_address_state,pmt_address_country,pmt_address_zip,pmt_address_landmark)) as address_id",
                                "pmt_address_line_1 as address_line_1",
                                "pmt_address_line_2 as address_line_2",
                                "pmt_address_line_3 as address_line_3",
                                "null as area",
                                "pmt_address_city as city",
                                "pmt_address_dist as district",
                                "pmt_address_state as state",
                                "pmt_address_country as country",
                                "pmt_address_zip as pin_code",
                                "pmt_address_landmark as landmark",
                                "case when pmt_address_is_verified is null then 'UV' else 'IV' end as verification_flag",
                                "case when pmt_address_is_verified is null then null else 'Mastercraft' end as verification_source",
                                "dc_action_date as last_updated",
                                "null as address_json"] 
                     }
              }
}

In [ ]:
def prep_data(spark_df,table_name, source, table_dict, prod_gpdb_spark_options, schema = "customermart"):
    from pyspark.sql import functions as F
    transform_expr = table_dict.get(table_name).get(source).get("transform_expr")
    if transform_expr == None:
        print("No Transformation found for this combination - {table_name},{source}".
              format(table_name=table_name,source=source))
        return None
    primary_key = table_dict.get(table_name).get("primary_key")
    date_sort_column = table_dict.get(table_name).get("date_sort_column")
    if (primary_key == None) | (date_sort_column == None) :
        print("No Primary Key OR Sort Key found for this table - {}".format(table_name))
        return None
    
    # each target table has a temp table with name table_name1
    # picking data from temp table (to get column names and their schema)
    
    df_1 = sqlContext.read.format("greenplum").options(**prod_gpdb_spark_options).option("dbschema",schema)\
    .option("dbtable","{}_staging".format(table_name)).load()
    dfschema = df_1.schema 
    
    from pyspark.sql import functions as F
    
    # applying transformations on spark_df
    spark_df = spark_df.selectExpr(transform_expr).select(*df_1.columns)
    final_df = sqlContext.createDataFrame(spark_df.rdd,dfschema).orderBy(F.desc("{}".format(date_sort_column))).drop_duplicates(primary_key).cache()
    records = final_df.count()
#     print(gpdb_spark_options)
    
#     inserting the data into temp table
    output_index = "address_staging"#customer_demographics_naddenk_01_07_22
    schema = "customermart"


    import sys
    try:

        final_df.write.option("truncate", "true").format("greenplum")\
        .option("dbtable",output_index).option('dbschema',schema)\
        .option("server.port","1150-1170").option("url",prod_url)\
        .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


    except Exception as e:
        x = e
        print(x)
    else:
        x = 200 #success
        print(x)
    return records
    #     print("{} records successfully Inserted in {}_staging!".format(records,table_name))

In [ ]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# This temp table has similar schema as original table but no json field and would not contain duplicates
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark

def update(conn_to,table_name,source,gpdb_spark_options = prod_gpdb_spark_options,
                  schema="customermart"):
    conn_to.rollback()
    update_fields = table_dict.get(table_name).get(source).get("update_fields")
    primary_key = table_dict.get(table_name).get("primary_key")
    
    # loading data from temp table to get column names
    df_1 = sqlContext.read.format("greenplum").options(**gpdb_spark_options).option("dbschema",schema)\
    .option("dbtable","{}_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col ]
    #out_columns = table_dict.get(table_name).get("out_columns")
#     records=df_1.count()

    cur_to = conn_to.cursor()  

    # this query insert records from temp table to target table when all the records in temp table are new and not present in target
    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.{table_name}_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    
    # this query will be executed to update the records that are already present in target, new values will be assigned from temp table
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    
    # this query will be executed after update query to insert the remaining records
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
        #update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
        #update_progress(table_name,source,time_filter,records)
    conn_to.commit()


In [ ]:
table_name = "address"
source = "permanent"

In [ ]:
latest_progress=get_latest_progress(table_name)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1  #currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:

col_str=['source_id','source_customer_id','dc_unified_id','psnt_address_line_1','psnt_address_line_2','psnt_address_line_3',
         'coalesce(pstincomingcity,psnt_address_city) as psnt_address_city',
         'coalesce(pstincomingstate,psnt_address_state) as psnt_address_state','psnt_address_country',
         'coalesce(pstincomingpin,psnt_address_zip) as psnt_address_zip','psnt_address_dist',
         'psnt_address_landmark','psnt_address_is_verified',
         'comm_address_line_1','comm_address_line_2','comm_address_line_3','coalesce(cmnincomingcity,comm_address_city) as comm_address_city' ,
         'comm_address_dist',
         'coalesce(cmnincomingstate,comm_address_state) as comm_address_state','comm_address_country',
         'coalesce(cmnincomingpin,comm_address_zip) as comm_address_zip','comm_address_landmark','comm_address_is_verified',
         'pmt_address_line_1','pmt_address_line_2','pmt_address_line_3','coalesce(pmtincomingcity,pmt_address_city) as pmt_address_city',
         'pmt_address_dist','coalesce(pmtincomingstate,pmt_address_state) as pmt_address_state',
         'pmt_address_country','coalesce(pmnincomingpin,pmt_address_zip) as pmt_address_zip',
         'pmt_address_landmark','pmt_address_is_verified', 'dc_action_date']

master_bulk_df=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod","dc_action_date",time_filter)
spark_df=master_bulk_df.filter((col('dc_unified_id').isNotNull()) & (col('source_id').isNotNull())) 

spark_df = spark_df.withColumn('psnt_address_zip',when(col('psnt_address_zip').rlike("^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"),\
                                                       col('psnt_address_zip')).otherwise(lit(None)))

spark_df = spark_df.withColumn('comm_address_zip',when(col('comm_address_zip').rlike("^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"),\
                                                       col('comm_address_zip')).otherwise(lit(None)))

spark_df = spark_df.withColumn('pmt_address_zip',when(col('pmt_address_zip').rlike("^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"),\
                                                       col('pmt_address_zip')).otherwise(lit(None)))


In [ ]:
spark_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
spark_df.count()

In [ ]:
if source =="permanent":

    df_per=spark_df.filter((col('pmt_address_city').isNotNull()) | (col('pmt_address_state').isNotNull()) | (col('pmt_address_zip').isNotNull()))
    if df_per.count()>0:
        records=prep_data(df_per,\
                  table_name,\
                  source,\
                  table_dict,\
                  prod_gpdb_spark_options
                 )
        conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
        conn_to=conn_prod
        import sys
        try:
            update(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart")
        except Exception as e:
            x = e
            print(x)
        else:
            x = 'success'
            print(x)
        if  x == 'success':
            update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
            print('SUCCESS')
        else:
            update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
            print('FAILED')
        conn_prod.close()

In [ ]:
table_name = "address"
source = "home"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
if source =="home":
    df_home=spark_df.filter((col('psnt_address_city').isNotNull()) | (col('psnt_address_state').isNotNull()) | (col('psnt_address_zip').isNotNull()))
    if df_home.count()>0:
        records=prep_data(df_home,\
                  table_name,\
                  source,\
                  table_dict,\
                  prod_gpdb_spark_options
                 )

        conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
        conn_to=conn_prod
        import sys
        try:
            update(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart")
        except Exception as e:
            x = e
            print(x)
        else:
            x = 'success'
            print(x)
        if  x == 'success':
            update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
            print('SUCCESS')
        else:
            update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
            print('FAILED')
        conn_prod.close()

In [ ]:
table_name = "address"
source = "communication"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
if source =="communication":
    df_com=spark_df.filter((col('comm_address_city').isNotNull()) | (col('comm_address_state').isNotNull()) | (col('comm_address_zip').isNotNull()))
    if df_com.count()>0:
        print(df_com.count())
        records=prep_data(df_com,\
                  table_name,\
                  source,\
                  table_dict,\
                  prod_gpdb_spark_options
                 )
        conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
        conn_to=conn_prod
        import sys
        try:
            update(conn_to,table_name,source,prod_gpdb_spark_options, schema="customermart")
        except Exception as e:
            x = e
            print(x)
        else:
            x = 'success'
            print(x)
        if  x == 'success':
            update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
            print('SUCCESS')
        else:
            update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
            print('FAILED')
        conn_prod.close()

In [ ]:
spark_df.unpersist()

In [ ]:
spark.stop()